In [1]:
import nltk # 자연어 툴킷
import threading # 단일 프로그램에서 경량(lightweight) 테스크를 생성하는 데 사용되는 스레딩 라이브러리
import queue # 멀티스레딩 프로그램에서 사용할 수 있는 대기열 라이브러리
import feedparser # RSS 피드 구문 분석(파싱) 라이브러리
import uuid # RFC-4122 기반 uuid 버전 1, 3, 4, 5 생성 라이브러리

In [2]:
threads = []  #프로그램의 모든 스레드를 추적하는 새로운 빈 리스트를 생성한다.
queues = [queue.Queue(), queue.Queue()] # 두 개의 대기열 목록을 만든다.
# 첫 번째 대기열 : 토큰화된 문장을 저장하는 데 사용된다.
# 두 번째 대기열 : 분석된 모든 품사 단어를 저장하는 데 사용된다.

In [3]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url) #parse() : url의 내용을 다운로드
    for entry in feed['entries'][:5]:
        text = entry['title'] #제목을 text에 저장
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text) #text를 분류하여 words에 저장
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [4]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

In [5]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            
            for path in chunks:
                try:
                    label = path.label() 
                    print(path, end=', ')
                except:
                    pass
            print()

In [6]:
def runProgram():
    
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()


In [7]:
if __name__ == '__main__':
    runProgram()


UnicodeEncodeError: 'utf-8' codec can't encode character '\ud83d' in position 0: surrogates not allowed